In [1]:
#load_ext line_profiler

import csv,operator,sys,os
import numpy as np
import sklearn
import json
import ReadData
import time
from functools import reduce

sys.path.append('../arch-forest/data/adult/')
sys.path.append('../arch-forest/data/bank/')
sys.path.append('../arch-forest/data/wine-quality/')
sys.path.append('../arch-forest/data/')
sys.path.append('../arch-forest/code/')
import trainForest
import Tree

import DecisionSnippetFeatures

dataPath = "/home/falkhoury/Study/Lab/Project/frequentTreesInRandomForests/arch-forest/data/"
frequentTreesPath = "/home/falkhoury/Study/Lab/Project/frequentTreesInRandomForests/forests/rootedFrequentTrees/"
resultsPath = "/home/falkhoury/Study/Lab/Project/frequentTreesInRandomForests/forests/rootedFrequentTrees/"

/home/falkhoury/anaconda3/envs/py3k/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
#import train data
from ReadData import readDataAdult,readWine, readWineTest
dataSet='adult'
if (dataSet == 'adult'):
    X_train,Y_train = ReadData.readDataAdult('train')
    X_test,Y_test = ReadData.readDataAdult('test')
if (dataSet == 'wine-quality'):
        X_train,Y_train = ReadData.readWine()
        #X_test,Y_test = ReadData.readWineTest()
print(len(X_train))
#print(len(X_test))
#red = np.genfromtxt("/home/falkhoury/Study/Lab/Project/frequentTreesInRandomForests/arch-forest/data/wine-quality/winequality-red.csv", delimiter=';', skip_header=1)
#white = np.genfromtxt("/home/falkhoury/Study/Lab/Project/frequentTreesInRandomForests/arch-forest/data/wine-quality/winequality-white.csv", delimiter=';', skip_header=1)
#X = np.vstack((red[:, :-1], white[:, :-1])).astype(dtype=np.float32)
#Y = np.concatenate((red[:, -1], white[:, -1]))
#Y = Y - min(Y)

32561


In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils.estimator_checks import check_estimator
from sklearn.metrics import accuracy_score

start_time = time.time()
model = DecisionTreeClassifier(max_depth=10)
dt_cv_score = cross_val_score(model, X_train, Y_train, cv=5, scoring='accuracy')
dt_time = time.time() - start_time
dt_score = dt_cv_score.mean()
print('DT: '+str(dt_score))
mymodel=Tree.Tree()
print(mymodel.str())

DT: 0.854918607313817


AttributeError: 'NoneType' object has no attribute 'prediction'

In [29]:
print(X_train[1])

[ 7.8     0.88    0.      2.6     0.098  25.     67.      0.9968  3.2
  0.68    9.8   ]


In [32]:
# grab adult training data in the format used by the random forests in forests/adult/text/

X = []
Y = []

f = open("/home/falkhoury/Study/Lab/Project/frequentTreesInRandomForests/arch-forest/data/adult/adult.data")
for row in f:
    if (len(row) > 1):
        entries = row.replace("\n", "").replace(" ", "").split(",")

        x = trainForest.getFeatureVector(entries)

        if (entries[-1] == "<=50K"):
            y = 0
        else:
            y = 1

        Y.append(y)
        X.append(x)

X = np.array(X).astype(dtype=np.int32)
Y = np.array(Y)
f.close()

In [13]:
print(X_train[2])

[    28      0      0      0      0      1      0      0      0 336951
      0      0      0      0      0      1      0      0      0      0
      0      0      0      0      0      0     12      1      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      1      0      0      0
      1      0      0      0      1      0      0      0      0      1
      0      0      0     40]


In [14]:
# load one of the readily available decision trees
f = open(dataPath+dataSet+'/text/DT_5.json')
dt = json.load(f)
f.close()


In [15]:
# this creates a transformation of the dataset, assigning each example the vertex id of the leaf vertex 
# the example turns out at. So far, so boring
fts = DecisionSnippetFeatures.FrequentSubtreeFeatures(dt).fit_transform(X[:20, :])
print(fts)

[[ 5]
 [51]
 [ 5]
 [42]
 [ 5]
 [ 6]
 [ 5]
 [43]
 [23]
 [50]
 [43]
 [50]
 [ 5]
 [18]
 [42]
 [33]
 [ 5]
 [ 8]
 [42]
 [18]]


# Something New, Something Interesting

Let's get funky.

For this, we need

- [ ] Frequent Patterns with split values
    - [x] New Data transformator JSON -> GRAPH
    - [x] Transform to GRAPH
    - [x] Mining frequent patterns ('Initial Rooted Frequent Subtree Mining (without embedding computation) -- With Split Values in Labels.ipynb')
    - [x] New Data transformator CSTRING -> JSON (cString2json.py updated)
    - [x] Transform to JSON ('Find All Occurrences of All Frequent Patterns of Size up to 6.ipynb')
    - [ ] fix missing member problem in SubtreeFeatures

In [16]:
rootedFrequentTrees = "RF_10_t16"
f = open(frequentTreesPath+dataSet+'/WithLeafEdges/leq6/'+rootedFrequentTrees+'.json')
frequentpatterns = json.load(f)
f.close()

JSONDecodeError: Unterminated string starting at: line 69 column 17 (char 4088)

In [6]:
%time dsf = DecisionSnippetFeatures.FrequentSubtreeFeatures(map(lambda x: x['pattern'], frequentpatterns[-100:]))

CPU times: user 1.56 ms, sys: 0 ns, total: 1.56 ms
Wall time: 1.65 ms


In [7]:
%time fts = dsf.fit_transform(X_train)

CPU times: user 12.6 s, sys: 0 ns, total: 12.6 s
Wall time: 12.6 s


In [8]:
from sklearn.preprocessing import OneHotEncoder
%time fts_onehot = OneHotEncoder(n_values=dsf.get_n_values()).fit_transform(fts)

CPU times: user 84.7 ms, sys: 10.7 ms, total: 95.5 ms
Wall time: 264 ms


/home/falkhoury/anaconda3/envs/py3k/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:373: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)] * n_features'.
  warnings.warn(msg, DeprecationWarning)


In [9]:
print(X_train.shape)
print(fts_onehot.shape)
print(fts.shape)
print(Y_train.shape)

(32561, 64)
(32561, 300)
(32561, 100)
(32561,)


# Classification Performance of Decision Tree Snippet Features vs. Normal Features 

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils.estimator_checks import check_estimator
from sklearn.metrics import accuracy_score

### Linear Regression

In [38]:

model = LinearRegression()

%time fts_onehot_cv_score = cross_val_score(model, fts_onehot, Y_train, cv=5, scoring='accuracy')
#model.fit(fts_onehot,Y_train)
#fts_onehot_cv_score = model.score(X_test,Y_test)
print(fts_onehot_cv_score)
#print(fts_onehot_cv_score.mean())
#print(Y)
#print(model.predict(X))
'''
model = LinearRegression()
model.fit(X_train,Y_train)
normalfeatures_cv_score = model.score(X_test,Y_test)
print(normalfeatures_cv_score)

%time normalfeatures_cv_score = cross_val_score(model, X, Y, cv=5, scoring='r2')
print(normalfeatures_cv_score)
print(normalfeatures_cv_score.mean())
#, scoring='neg_mean_squared_error'

model = DecisionTreeClassifier(max_depth=15)
%time normalfeatures_dt_cv_score = cross_val_score(model, X, Y, cv=5, scoring='neg_mean_squared_error')
print(normalfeatures_dt_cv_score)

model = RandomForestClassifier(max_depth=15, n_estimators=100)
%time normalfeatures_rf_cv_score = cross_val_score(model, X, Y, cv=5, scoring='neg_mean_squared_error')
print(normalfeatures_rf_cv_score)
#f1

with open(resultsPath+dataSet+'/'+rootedFrequentTrees+'.txt', 'w') as f:
        f.write(fts_onehot_cv_score.mean()+'/n'+normalfeatures_cv_score.mean()+'/n'+normalfeatures_dt_cv_score+'/n'+normalfeatures_rf_cv_score)
f.close()'''

ValueError: Classification metrics can't handle a mix of binary and continuous targets

[0.34063762 0.32159788 0.3349055  0.35143024 0.350273  ]


"\nmodel = LinearRegression()\nmodel.fit(X_train,Y_train)\nnormalfeatures_cv_score = model.score(X_test,Y_test)\nprint(normalfeatures_cv_score)\n\n%time normalfeatures_cv_score = cross_val_score(model, X, Y, cv=5, scoring='r2')\nprint(normalfeatures_cv_score)\nprint(normalfeatures_cv_score.mean())\n#, scoring='neg_mean_squared_error'\n\nmodel = DecisionTreeClassifier(max_depth=15)\n%time normalfeatures_dt_cv_score = cross_val_score(model, X, Y, cv=5, scoring='neg_mean_squared_error')\nprint(normalfeatures_dt_cv_score)\n\nmodel = RandomForestClassifier(max_depth=15, n_estimators=100)\n%time normalfeatures_rf_cv_score = cross_val_score(model, X, Y, cv=5, scoring='neg_mean_squared_error')\nprint(normalfeatures_rf_cv_score)\n#f1\n\nwith open(resultsPath+dataSet+'/'+rootedFrequentTrees+'.txt', 'w') as f:\n        f.write(fts_onehot_cv_score.mean()+'/n'+normalfeatures_cv_score.mean()+'/n'+normalfeatures_dt_cv_score+'/n'+normalfeatures_rf_cv_score)\nf.close()"

In [24]:
model = LinearRegression()
%time normalfeatures_cv_score = cross_val_score(model, X, Y, cv=5, scoring='neg_mean_squared_error')
print(normalfeatures_cv_score)

CPU times: user 28.1 ms, sys: 0 ns, total: 28.1 ms
Wall time: 15.6 ms
[-0.55084091 -0.63107877 -0.60470598 -0.56539122 -0.52256548]


### Naive Bayes

In [11]:
# Classification
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils.estimator_checks import check_estimator
from sklearn.metrics import accuracy_score
    
results_list = []
time_list = []
rf_depth = 5
scoring_function = 'accuracy'
pattern_max_size=6
variant = 'NoLeafEdges'

start_time = time.time()
model = GaussianNB()
normalfeatures_nb_cv_score = cross_val_score(model, X_train, Y_train, cv=5, scoring=scoring_function)
normal_time = time.time() - start_time
normal_score = normalfeatures_nb_cv_score.mean()
print('normal: '+str(normal_score))

start_time = time.time()
model = DecisionTreeClassifier(max_depth=rf_depth)
dt_cv_score = cross_val_score(model, X_train, Y_train, cv=5, scoring=scoring_function)
dt_time = time.time() - start_time
dt_score = dt_cv_score.mean()
print('DT: '+str(dt_score))
    
start_time = time.time()    
model = RandomForestClassifier(max_depth=rf_depth, n_estimators=100)
rf_cv_score = cross_val_score(model, X_train, Y_train, cv=5, scoring=scoring_function)
rf_time = time.time() - start_time
rf_score = rf_cv_score.mean()
print('RF: '+str(rf_score))

for frequency in range(2,26):
    start_time = time.time()
    rootedFrequentTrees = "RF_"+str(rf_depth)+"_t"+str(frequency)
    f = open(frequentTreesPath+dataSet+'/'+variant+'/leq'+str(pattern_max_size)+'/'+rootedFrequentTrees+'.json')
    frequentpatterns = json.load(f)
    f.close()

    dsf = DecisionSnippetFeatures.FrequentSubtreeFeatures(map(lambda x: x['pattern'], frequentpatterns[-100:]))
    

    fts = dsf.fit_transform(X_train)

    from sklearn.preprocessing import OneHotEncoder
    fts_onehot = OneHotEncoder(n_values=dsf.get_n_values()).fit_transform(fts)

    



    model = GaussianNB()
    fts_onehot_nb_cv_score = cross_val_score(model, fts_onehot.toarray(), Y_train, cv=5, scoring=scoring_function)
    dsf_time = time.time() - start_time
    print(fts_onehot_nb_cv_score)
    dsf_score = fts_onehot_nb_cv_score.mean()
    print('t'+str(frequency )+'_DSF: '+str(dsf_score))
    
    
    
    result = "t"+str(frequency)+","+str(dsf_score)+","+str(normal_score)+","+str(dt_score)+","+str(rf_score)+",\n"
    results_list.append(result)
    times = "t"+str(frequency)+","+str(dsf_time)+","+str(normal_time)+","+str(dt_time)+","+str(rf_time)+",\n"
    time_list.append(times)

f= open(resultsPath+dataSet+'/Results_'+variant+'/leq'+str(pattern_max_size)+'/'+'RF_'+str(rf_depth)+'_'+scoring_function+'.csv',"a")
for line in results_list:
    f.write(line)
f.close()

f= open(resultsPath+dataSet+'/Results_'+variant+'/leq'+str(pattern_max_size)+'/'+'RF_'+str(rf_depth)+'_'+scoring_function+'_time.csv',"a")
for line in time_list:
    f.write(line)
f.close()

normal: 0.7951537437315881
DT: 0.8478549846064816
RF: 0.8440466779538636


/home/falkhoury/anaconda3/envs/py3k/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:373: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)] * n_features'.
  warnings.warn(msg, DeprecationWarning)


[0.8397052  0.84244472 0.84382678 0.84566953 0.84398034]
t2_DSF: 0.8431253161792084


/home/falkhoury/anaconda3/envs/py3k/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:373: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)] * n_features'.
  warnings.warn(msg, DeprecationWarning)


[0.83463842 0.83338452 0.84075553 0.83875921 0.84305897]
t3_DSF: 0.838119329286994


...

/home/falkhoury/anaconda3/envs/py3k/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:373: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)] * n_features'.
  warnings.warn(msg, DeprecationWarning)
/home/falkhoury/anaconda3/envs/py3k/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:373: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)] * n_features'.
  warnings.warn(msg, DeprecationWarning)


[0.68601259 0.67644349 0.68074324 0.67536855 0.67659705]
t18_DSF: 0.6790329848713083
[0.71672041 0.70608108 0.70961302 0.71160934 0.70746314]
t19_DSF: 0.7102973980219488


/home/falkhoury/anaconda3/envs/py3k/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:373: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)] * n_features'.
  warnings.warn(msg, DeprecationWarning)
/home/falkhoury/anaconda3/envs/py3k/lib/python3.7/site-packages/sklearn/naive_bayes.py:434: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
/home/falkhoury/anaconda3/envs/py3k/lib/python3.7/site-packages/sklearn/naive_bayes.py:436: RuntimeWarning: invalid value encountered in true_divide
  (self.sigma_[i, :]), 1)
/home/falkhoury/anaconda3/envs/py3k/lib/python3.7/site-packages/sklearn/naive_bayes.py:434: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
/home/falkhoury/anaconda3/envs/py3k/lib/python3.7/site-pac

[0.75909719 0.75921376 0.75921376 0.75921376 0.75921376]
t20_DSF: 0.7591904454179904
[0.75909719 0.75921376 0.75921376 0.75921376 0.75921376]
t21_DSF: 0.7591904454179904
[0.75909719 0.75921376 0.75921376 0.75921376 0.75921376]
t22_DSF: 0.7591904454179904
[0.75909719 0.75921376 0.75921376 0.75921376 0.75921376]
t23_DSF: 0.7591904454179904
[0.75909719 0.75921376 0.75921376 0.75921376 0.75921376]
t24_DSF: 0.7591904454179904
[0.75909719 0.75921376 0.75921376 0.75921376 0.75921376]
t25_DSF: 0.7591904454179904


/home/falkhoury/anaconda3/envs/py3k/lib/python3.7/site-packages/sklearn/naive_bayes.py:434: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
/home/falkhoury/anaconda3/envs/py3k/lib/python3.7/site-packages/sklearn/naive_bayes.py:436: RuntimeWarning: invalid value encountered in true_divide
  (self.sigma_[i, :]), 1)
/home/falkhoury/anaconda3/envs/py3k/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:373: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)] * n_features'.
  warnings.warn(msg, DeprecationWarning)
/home/falkhoury/anaconda3/envs/py3k/lib/python3.7/site-packages/sklearn/naive_bayes.py:434: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
/home/falkhoury/anaconda3/envs/py3k/lib/python3.7/site-pac

In [33]:
# Regression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils.estimator_checks import check_estimator
from sklearn.metrics import accuracy_score
    
results_list = []
time_list = []
rf_depth = 20
scoring_function_Reg = 'neg_mean_squared_error'
scoring_function_Class = 'accuracy'
pattern_max_size=6

start_time = time.time()
model = LinearRegression()
normalfeatures_nb_cv_score = cross_val_score(model, X_train, Y_train, cv=5, scoring=scoring_function_Reg)
normal_time = time.time() - start_time
normal_score = normalfeatures_nb_cv_score.mean()
print('normal: '+str(normal_score))

start_time = time.time()
model = DecisionTreeClassifier(max_depth=rf_depth)
dt_cv_score = cross_val_score(model, X_train, Y_train, cv=5, scoring=scoring_function_Class)
dt_time = time.time() - start_time
dt_score = dt_cv_score.mean()
print('DT: '+str(dt_score))
    
start_time = time.time()    
model = RandomForestClassifier(max_depth=rf_depth, n_estimators=100)
rf_cv_score = cross_val_score(model, X_train, Y_train, cv=5, scoring=scoring_function_Class)
rf_time = time.time() - start_time
rf_score = rf_cv_score.mean()
print('RF: '+str(rf_score))

for frequency in range(2,26):
    start_time = time.time()
    rootedFrequentTrees = "RF_"+str(rf_depth)+"_t"+str(frequency)
    f = open(frequentTreesPath+dataSet+'/WithLeafEdges/leq'+str(pattern_max_size)+'/'+rootedFrequentTrees+'.json')
    frequentpatterns = json.load(f)
    f.close()

    dsf = DecisionSnippetFeatures.FrequentSubtreeFeatures(map(lambda x: x['pattern'], frequentpatterns[-100:]))
    

    fts = dsf.fit_transform(X_train)

    from sklearn.preprocessing import OneHotEncoder
    fts_onehot = OneHotEncoder(n_values=dsf.get_n_values()).fit_transform(fts)

    



    model = LinearRegression()
    fts_onehot_nb_cv_score = cross_val_score(model, fts_onehot.toarray(), Y_train, cv=5, scoring=scoring_function_Reg)
    dsf_time = time.time() - start_time
    print(fts_onehot_nb_cv_score)
    dsf_score = fts_onehot_nb_cv_score.mean()
    print('t'+str(frequency )+'_DSF: '+str(dsf_score))
    
    
    
    result = "t"+str(frequency)+","+str(dsf_score)+","+str(normal_score)+","+str(dt_score)+","+str(rf_score)+",\n"
    results_list.append(result)
    times = "t"+str(frequency)+","+str(dsf_time)+","+str(normal_time)+","+str(dt_time)+","+str(rf_time)+",\n"
    time_list.append(times)

f= open(resultsPath+dataSet+'/leq'+str(pattern_max_size)+'/'+'RF_'+str(rf_depth)+'_'+scoring_function+'.csv',"a")
for line in results_list:
    f.write(line)
f.close()

f= open(resultsPath+dataSet+'/leq'+str(pattern_max_size)+'/'+'RF_'+str(rf_depth)+'_'+scoring_function+'_time.csv',"a")
for line in time_list:
    f.write(line)
f.close()

normal: -0.5749164708558536
DT: 0.3875821018733852
RF: 0.476892449686117


/home/falkhoury/anaconda3/envs/py3k/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:373: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)] * n_features'.
  warnings.warn(msg, DeprecationWarning)


[-1.50695572e+20 -6.15893276e-01 -6.23840990e-01 -5.73100056e-01
 -5.20318588e-01]
t2_DSF: -3.013911440957144e+19


/home/falkhoury/anaconda3/envs/py3k/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:373: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)] * n_features'.
  warnings.warn(msg, DeprecationWarning)


[-8.05755334e-01 -6.42845588e-01 -6.46236147e-01 -8.03289039e+22
 -5.36516867e-01]
t3_DSF: -1.606578078964174e+22


...

/home/falkhoury/anaconda3/envs/py3k/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:373: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)] * n_features'.
  warnings.warn(msg, DeprecationWarning)


[-0.82990027 -0.69060309 -0.6882788  -0.67817576 -0.52632717]
t22_DSF: -0.68265701543221


/home/falkhoury/anaconda3/envs/py3k/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:373: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)] * n_features'.
  warnings.warn(msg, DeprecationWarning)


[-0.81261813 -0.67338982 -0.6877939  -0.68213938 -0.52937482]
t23_DSF: -0.6770632094329915
[-0.56000189 -0.70971251 -0.71696193 -0.6813558  -0.56368941]
t24_DSF: -0.6463443103198152
[-0.67442866 -0.8081986  -0.80165699 -0.86750121 -0.66714072]
t25_DSF: -0.7637852374943163


/home/falkhoury/anaconda3/envs/py3k/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:373: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)] * n_features'.
  warnings.warn(msg, DeprecationWarning)
/home/falkhoury/anaconda3/envs/py3k/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:373: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)] * n_features'.
  warnings.warn(msg, DeprecationWarning)


In [15]:
model = GaussianNB()
%time normalfeatures_nb_cv_score = cross_val_score(model, X_train, Y_train, cv=5, scoring='accuracy')
print(normalfeatures_nb_cv_score)
print(normalfeatures_nb_cv_score.mean())

CPU times: user 323 ms, sys: 7.87 ms, total: 330 ms
Wall time: 176 ms
[0.7941041  0.79176904 0.79867936 0.79130835 0.79990786]
0.7951537437315881


In [23]:
#with open(resultsPath+dataSet+'/'+rootedFrequentTrees+'.txt', 'w+') as f:
#        f.write('test2')
#f.close()

f= open(resultsPath+dataSet+'/'+rootedFrequentTrees+'.txt',"a")
f.write('test3')
f.close()
#model.fit(fts_onehot.toarray(),Y_train)
#model.predict(X_test[722].reshape(-1, 1))

#print(model.score(X_test,Y_test))

### Thresholded Linear Regression

In [35]:
class LinRegClassifier(LinearRegression):
    def __init__(self, threshold=0.5):
        super(LinRegClassifier, self).__init__()
        self.threshold = threshold
    
    def predict(self, X):
        p = super(LinRegClassifier, self).predict(X)
        return (p > self.threshold)

check_estimator(LinRegClassifier)

/home/falkhoury/anaconda3/envs/py3k/lib/python3.7/site-packages/sklearn/utils/estimator_checks.py:306: SkipTestWarning: pandas is not installed: not testing for input of type pandas.Series to class weight.
  warnings.warn(str(exception), SkipTestWarning)


AssertionError: 0.3294345956863125 not greater than 0.5

In [28]:
model = LinRegClassifier()
%time fts_onehot_nb_cv_score = cross_val_score(model, fts_onehot, Y, cv=5, scoring='f1')
print(fts_onehot_nb_cv_score)

NameError: name 'LinRegClassifier' is not defined

In [37]:
model = LinRegClassifier()
%time normalfeatures_nb_cv_score = cross_val_score(model, X, Y, cv=5, scoring='f1')
print(normalfeatures_nb_cv_score)

CPU times: user 734 ms, sys: 323 ms, total: 1.06 s
Wall time: 307 ms
[0.58562691 0.59600614 0.60770111 0.59946133 0.61109044]


## Comparison to DT and RF on Train

In [17]:
model = DecisionTreeClassifier(max_depth=15)
%time normalfeatures_nb_cv_score = cross_val_score(model, X, Y, cv=5, scoring='f1')
print(normalfeatures_nb_cv_score)

CPU times: user 895 ms, sys: 3.84 ms, total: 899 ms
Wall time: 898 ms
[0.63962454 0.64781145 0.661875   0.68265068 0.66861598]


In [18]:
model = RandomForestClassifier(max_depth=15, n_estimators=100)
%time normalfeatures_nb_cv_score = cross_val_score(model, X, Y, cv=5, scoring='f1')
print(normalfeatures_nb_cv_score)
#f1

CPU times: user 10 s, sys: 11.4 ms, total: 10 s
Wall time: 10 s
[0.65958226 0.66120015 0.67824828 0.68050408 0.68066618]


## Comparison of LinearRegression on DT features -> Should be more or less identical

In [42]:
# load one of the readily available decision trees
f = open('/home/falkhoury/Study/Lab/Project/frequentTreesInRandomForests/arch-forest/data/adult/text/DT_5.json')
dt = json.load(f)
f.close()
dt_fsf = DecisionSnippetFeatures.FrequentSubtreeFeatures(dt)
dt_fts = dt_fsf.fit_transform(X)
dt_fts_onehot = OneHotEncoder(n_values=dt_fsf.get_n_values()).fit_transform(dt_fts)
print(dt_fts_onehot.shape)

(32561, 59)


/home/falkhoury/anaconda3/envs/py3k/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:373: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)] * n_features'.
  warnings.warn(msg, DeprecationWarning)


In [45]:
from sklearn.base import BaseEstimator, ClassifierMixin
class UntrainableDTClassifier(BaseEstimator):
    def __init__(self, dt):
        super(UntrainableDTClassifier, self).__init__()
        self.decisionTreeModel = DecisionSnippetFeatures.FrequentSubtreeFeatures(dt)

        tree = self.decisionTreeModel.patterns[0]
        self.linreg_weights = np.zeros(self.decisionTreeModel.get_n_values())
        for i in range(self.decisionTreeModel.get_n_values()):
            try:
                self.linreg_weights[i] = tree.nodes[i].prediction[0]
            except TypeError:
                self.linreg_weights[i] = 0
        
    def fit(self, X, y):
        pass

    def predict(self, X):
        return np.dot(X, self.linreg_weights)

check_estimator(UntrainableDTClassifier)
    
    
model = UntrainableDTClassifier(dt)
print(dt_fts_onehot.shape)
%time 
dtfeatures_lr_cv_score = cross_val_score(model, dt_fts_onehot, Y, cv=5, scoring='f1')
print(dtfeatures_lr_cv_score)

TypeError: __init__() missing 1 required positional argument: 'dt'

In [46]:
a = np.array([1,2,3])
b = np.array([[1,2,3], [1,2,3]])
print(a)
print(b)
print(np.dot(b,a))

[1 2 3]
[[1 2 3]
 [1 2 3]]
[14 14]


## Comparison of NaiveBayes on DT features -> Should be more or less identical

In [40]:
# load one of the readily available decision trees
f = open('/home/falkhoury/Study/Lab/Project/frequentTreesInRandomForests/arch-forest/data/adult/text/DT_5.json')
dt = json.load(f)
f.close()
dt_fsf = DecisionSnippetFeatures.FrequentSubtreeFeatures(dt)
dt_fts = dt_fsf.fit_transform(X)
# dt_fts_onehot = OneHotEncoder(n_values=dt_fsf.get_n_values()).fit_transform(fts)
print(dt_fsf.get_n_values())

IndexError: index 50 is out of bounds for axis 0 with size 11

In [39]:
model = GaussianNB()
%time dtfeatures_nb_cv_score = cross_val_score(model, dt_fts, Y, cv=5, scoring='neg_mean_squared_error')
print(dtfeatures_nb_cv_score)

NameError: name 'dt_fts' is not defined

NameError: name 'dtfeatures_nb_cv_score' is not defined